In [5]:
from pyspark.sql import SparkSession
spark = (SparkSession.builder.appName("cs544")
         .config("spark.sql.shuffle.partitions", 10)
         .config("spark.ui.showConsoleProgress", False)
         .config('spark.jars.packages', 'org.apache.spark:spark-sql-kafka-0-10_2.12:3.2.2')
         .getOrCreate())

df = (
    spark.readStream.format("kafka")
    .option("kafka.bootstrap.servers", "kafka:9092")
    .option("subscribe", "stations-json")
    .option("startingOffsets", "earliest")
    .load()
)

:: loading settings :: url = jar:file:/usr/local/lib/python3.10/dist-packages/pyspark/jars/ivy-2.5.0.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /root/.ivy2/cache
The jars for the packages stored in: /root/.ivy2/jars
org.apache.spark#spark-sql-kafka-0-10_2.12 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-438a130f-c129-4b31-849a-0de41fefe7e5;1.0
	confs: [default]
	found org.apache.spark#spark-sql-kafka-0-10_2.12;3.2.2 in central
	found org.apache.spark#spark-token-provider-kafka-0-10_2.12;3.2.2 in central
	found org.apache.kafka#kafka-clients;2.8.1 in central
	found org.lz4#lz4-java;1.7.1 in central
	found org.xerial.snappy#snappy-java;1.1.8.4 in central
	found org.slf4j#slf4j-api;1.7.30 in central
	found org.apache.hadoop#hadoop-client-runtime;3.3.1 in central
	found org.spark-project.spark#unused;1.0.0 in central
	found org.apache.hadoop#hadoop-client-api;3.3.1 in central
	found org.apache.htrace#htrace-core4;4.1.0-incubating in central
	found commons-logging#commons-logging;1.1.3 in central
	found com.google.code.findbugs#jsr305;3.0.0 in central
	found org.ap

In [6]:
df.isStreaming

True

In [7]:
from pyspark.sql.types import StructType, StructField, StringType, DateType, DoubleType, BooleanType
from pyspark.sql.functions import col, from_json

schema = StructType([
    StructField("station", StringType()),
    StructField("date", DateType()),
    StructField("degrees", DoubleType()),
    StructField("raining", BooleanType())
])

reports = (df.select(from_json(col("value").cast("string"), schema).alias("value")).select("value.*"))

In [8]:
reports.isStreaming

True

In [9]:
reports.printSchema()

root
 |-- station: string (nullable = true)
 |-- date: date (nullable = true)
 |-- degrees: double (nullable = true)
 |-- raining: boolean (nullable = true)



In [10]:
from pyspark.sql.functions import *

counts_df = reports.groupBy("station") \
                  .agg(min("date").alias("start"),
                       max("date").alias("end"),
                       count("degrees").alias("measurements"),
                       avg("degrees").alias("avg"),
                       max("degrees").alias("max"))

In [11]:
s = counts_df.writeStream.format("console").trigger(processingTime="5 seconds").outputMode("complete").start()
s.awaitTermination(30)
s.stop()

23/04/27 00:22:48 WARN ResolveWriteToStream: Temporary checkpoint location created which is deleted normally when the query didn't fail: /tmp/temporary-e071b235-2c9d-46f7-b896-4b13454419ea. If it's required to delete it under any circumstances, please set spark.sql.streaming.forceDeleteTempCheckpointLocation to true. Important to know deleting temp checkpoint folder is best effort.
23/04/27 00:22:48 WARN ResolveWriteToStream: spark.sql.adaptive.enabled is not supported in streaming DataFrames/Datasets and will be disabled.


-------------------------------------------
Batch: 0
-------------------------------------------
+-------+----------+----------+------------+------------------+------------------+
|station|     start|       end|measurements|               avg|               max|
+-------+----------+----------+------------+------------------+------------------+
|      F|2000-01-01|2000-06-23|         175| 45.06726264502637| 86.07853676828634|
|      K|2000-01-01|2000-06-23|         175| 44.03214643986366| 83.34679494196683|
|      I|2000-01-01|2000-06-23|         175| 53.22453770414406|  98.2338390215053|
|      N|2000-01-01|2000-06-23|         175| 63.67271399123035| 101.4117221949113|
|      E|2000-01-01|2000-06-23|         175| 51.09822956020594| 94.61485479137116|
|      J|2000-01-01|2000-06-23|         175| 55.55086535844712| 90.90854232938617|
|      A|2000-01-01|2000-06-23|         175| 61.95068403070622| 99.95112559927841|
|      H|2000-01-01|2000-06-23|         175|52.317780607746826| 91.219013

23/04/27 00:23:03 WARN ProcessingTimeExecutor: Current batch is falling behind. The trigger interval is 5000 milliseconds, but spent 14282 milliseconds


-------------------------------------------
Batch: 1
-------------------------------------------
+-------+----------+----------+------------+------------------+------------------+
|station|     start|       end|measurements|               avg|               max|
+-------+----------+----------+------------+------------------+------------------+
|      K|2000-01-01|2000-07-01|         183| 45.60382793068246| 89.97203943719316|
|      F|2000-01-01|2000-07-01|         183| 46.44856722640371| 86.07853676828634|
|      I|2000-01-01|2000-07-01|         183|54.525033130128996|  98.2338390215053|
|      N|2000-01-01|2000-07-01|         183| 64.81192653192767| 101.4117221949113|
|      E|2000-01-01|2000-07-01|         183|51.959158584743264| 94.61485479137116|
|      J|2000-01-01|2000-07-01|         183| 56.85985679789013| 90.90854232938617|
|      A|2000-01-01|2000-07-01|         183|  62.9432726844083| 99.95112559927841|
|      H|2000-01-01|2000-07-01|         183|53.290360260647525| 91.219013

In [12]:
reports.printSchema()
reports

root
 |-- station: string (nullable = true)
 |-- date: date (nullable = true)
 |-- degrees: double (nullable = true)
 |-- raining: boolean (nullable = true)



In [19]:
today = reports.select(
    when(col("raining") == 1, 1).otherwise(0).alias("raining"),
    col("station").cast("string").alias("station"),
    col("date").cast("date").alias("date")
)

In [61]:
from pyspark.sql.window import Window

features = reports.select(
    col("station").alias("station"),
    col("date").alias("date"),
    month("date").alias("month"),
    lag("degrees", 1).over(Window.partitionBy("station").orderBy("date")).alias("sub1degrees"),
    lag("raining", 1).over(Window.partitionBy("station").orderBy("date")).alias("sub1raining"),
    lag("degrees", 2).over(Window.partitionBy("station").orderBy("date")).alias("sub2degrees"),
    lag("raining", 2).over(Window.partitionBy("station").orderBy("date")).alias("sub2raining")
)



In [62]:
features.printSchema()

root
 |-- station: string (nullable = true)
 |-- date: date (nullable = true)
 |-- month: integer (nullable = true)
 |-- sub1degrees: double (nullable = true)
 |-- sub1raining: boolean (nullable = true)
 |-- sub2degrees: double (nullable = true)
 |-- sub2raining: boolean (nullable = true)



In [63]:
k=features.join(today, ["date", "station"])

In [64]:
stream_query=(k
              .repartition(1)
              .writeStream
              .format("parquet")
              .option("path", "output_parquet")
              .option("checkpointLocation", "checkpoint")
              .trigger(processingTime="1 minute")
              .start())

stream_query.awaitTermination(61)
stream_query.stop()

23/04/27 01:04:17 WARN ResolveWriteToStream: spark.sql.adaptive.enabled is not supported in streaming DataFrames/Datasets and will be disabled.


AnalysisException: Non-time-based windows are not supported on streaming DataFrames/Datasets;
Window [lag(degrees#25, -1, null) windowspecdefinition(station#23, date#24 ASC NULLS FIRST, specifiedwindowframe(RowFrame, -1, -1)) AS sub1degrees#613, lag(raining#26, -1, null) windowspecdefinition(station#23, date#24 ASC NULLS FIRST, specifiedwindowframe(RowFrame, -1, -1)) AS sub1raining#614, lag(degrees#25, -2, null) windowspecdefinition(station#23, date#24 ASC NULLS FIRST, specifiedwindowframe(RowFrame, -2, -2)) AS sub2degrees#615, lag(raining#26, -2, null) windowspecdefinition(station#23, date#24 ASC NULLS FIRST, specifiedwindowframe(RowFrame, -2, -2)) AS sub2raining#616], [station#23], [date#24 ASC NULLS FIRST]
+- Project [station#23 AS station#610, date#24 AS date#611, month(date#24) AS month#612, degrees#25, station#23, date#24, raining#26]
   +- Project [value#21.station AS station#23, value#21.date AS date#24, value#21.degrees AS degrees#25, value#21.raining AS raining#26]
      +- P

In [86]:
# Define the features DataFrame
features_yesterday = reports.select(
  col("station"),
  date_add(col("date"), -1).alias("date"),
  col("raining").alias("sub1raining"),
  col("degrees").alias("sub1degrees"),
)
features_two_days_ago = reports.select(
  col("station"),
  date_add(col("date"), -2).alias("date"),
  col("degrees").alias("sub2degrees"),
  col("raining").alias("sub1raining"),
)
features = features_yesterday.join(
  features_two_days_ago,
  ["station", "date"],
  "inner"
).withColumn("month", col("date").substr(6, 2).cast("int"))

# Join today with features on the station, date, and month columns
result = today.join(features, ["station", "date"])

# Write the results to parquet files
stream_query=(result
              .repartition(1)
              .writeStream
              .format("parquet")
              .option("path", "output_parquet")
              .option("checkpointLocation", "checkpoint")
              .trigger(processingTime="1 minute")
              .start())

stream_query.awaitTermination(61)
stream_query.stop()

23/04/27 01:40:46 WARN ResolveWriteToStream: spark.sql.adaptive.enabled is not supported in streaming DataFrames/Datasets and will be disabled.
23/04/27 01:40:49 WARN HDFSBackedStateStoreProvider: The state for version 2 doesn't exist in loadedMaps. Reading snapshot file and delta files if needed...Note that this is normal for the first batch of starting query.
23/04/27 01:40:49 WARN HDFSBackedStateStoreProvider: The state for version 2 doesn't exist in loadedMaps. Reading snapshot file and delta files if needed...Note that this is normal for the first batch of starting query.
23/04/27 01:40:49 ERROR StateSchemaCompatibilityChecker: Provided schema doesn't match to the schema for existing state! Please note that Spark allow difference of field name: check count of fields and data type of each field.
- Provided key schema: StructType(StructField(field0,StringType,true), StructField(field1,DateType,true), StructField(index,LongType,true))
- Existing key schema: StructType(StructField(fie

StreamingQueryException: Job aborted.
=== Streaming Query ===
Identifier: [id = 304e2c9e-3e96-485d-bf57-fc397db6810b, runId = 9f45308e-d8ec-45d6-9849-6e267fee1b9d]
Current Committed Offsets: {KafkaV2[Subscribe[stations-json]]: {"stations-json":{"2":12069,"5":4023,"4":20115,"1":8046,"3":12069,"0":4023}}}
Current Available Offsets: {KafkaV2[Subscribe[stations-json]]: {"stations-json":{"2":13758,"5":4586,"4":22930,"1":9172,"3":13758,"0":4586}}}

Current State: ACTIVE
Thread State: RUNNABLE

Logical Plan:
Repartition 1, true
+- Project [station#358, date#359, raining#357, sub1raining#1473, sub1degrees#1474, sub2degrees#1480, sub1raining#1481, month#1496]
   +- Join Inner, ((station#358 = station#23) AND (date#359 = date#1472))
      :- Project [CASE WHEN raining#26 THEN 1 ELSE 0 END AS raining#357, cast(station#23 as string) AS station#358, cast(date#24 as date) AS date#359]
      :  +- Project [value#21.station AS station#23, value#21.date AS date#24, value#21.degrees AS degrees#25, value

In [79]:
stream_query.stop()

In [80]:
data=spark.read.parquet("output_parquet")

In [81]:
data.count()

60315

In [83]:
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.classification import DecisionTreeClassifier
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

assembler = VectorAssembler(inputCols=["month", "sub1degrees", "sub1raining", "sub2degrees", "sub2raining"], outputCol="features")
data = assembler.transform(data)

IllegalArgumentException: sub1degrees does not exist. Available: station, date, raining, sub1raining, sub2raining, month